In [1]:
import os
import modin.pandas as pd

def clean_gff_file(file_path, output_directory):
    try:
        # Read the dataset
        df = pd.read_csv(file_path, sep='\t', header=None)

        # Check if column 8 exists and is not entirely empty
        if 8 < len(df.columns) and df[8].notna().any():
            try:
                # Split column 8 into separate columns
                split_data = df[8].str.split(';', expand=True)
                split_data = split_data.fillna('')
                split_data = split_data.applymap(lambda x: x.split('=', 1)[-1] if x else '')

                # Check if split_data has the expected number of columns
                if len(split_data.columns) == 7:
                    split_data.columns = ['activity', 'bound_end', 'bound_start', 'description', 'epigenome', 'feature_type', 'regulatory_feature_stable_id']

                    # Concatenate the split data with the original DataFrame
                    df = pd.concat([df.iloc[:, :8], split_data], axis=1)

                    # Remove null or repeating values in the regulatory_feature_stable_id column
                    df = df[df['regulatory_feature_stable_id'].notnull()]
                    df = df.drop_duplicates(subset=['regulatory_feature_stable_id'])

                    # Define the column names
                    column_names = ['chromosome', 'regulatory_build', 'regulatory_feature', 'chromosome_start_site', 'chromosome_end_site', 'score', 'strand', 'phase', 'activity', 'bound_end', 'bound_start', 'description', 'epigenome', 'feature_type', 'regulatory_feature_stable_id']
                    df.columns = column_names

                    # Save the updated DataFrame to a new file in the output directory
                    output_file_path = os.path.join(output_directory, os.path.basename(file_path).replace('.gff', '_cleaned.gff'))
                    df.to_csv(output_file_path, sep='\t', index=False, header=True)
                else:
                    print(f"Unexpected number of columns after splitting column 8: {file_path}")
            except Exception as e:
                print(f"Error splitting column 8: {file_path}, {e}")
        else:
            print(f"Column 8 does not exist or is entirely empty: {file_path}")
    except pd.errors.EmptyDataError:
        print(f"Empty or malformed file: {file_path}")

def clean_gff_files_in_directory(input_directory, output_directory):
    for root, dirs, files in os.walk(input_directory):
        for file in files:
            if file.endswith(".gff"):
                file_path = os.path.join(root, file)
                print(f"Cleaning: {file_path}")
                clean_gff_file(file_path, output_directory)

# Replace "input_directory" with the path to the directory containing the subfolders
input_directory = "/data/project/ubrite/drg-depot/Ensembl/rawdata/homo_sapiens/RegulatoryFeatureActivity/RegulatoryFeatureActivity/"

# Replace "output_directory" with the path to a different directory where you want to move the cleaned files
output_directory = "/data/project/ubrite/drg-depot/Ensembl/cleaned_data/Regulatory_feature_activity/"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Clean GFF files in the input directory
clean_gff_files_in_directory(input_directory, output_directory)


Cleaning: /data/project/ubrite/drg-depot/Ensembl/rawdata/homo_sapiens/RegulatoryFeatureActivity/RegulatoryFeatureActivity/ovary/homo_sapiens.GRCh38.ovary.Regulatory_Build.regulatory_activity.20221007.gff
Cleaning: /data/project/ubrite/drg-depot/Ensembl/rawdata/homo_sapiens/RegulatoryFeatureActivity/RegulatoryFeatureActivity/H1_hESC_3/homo_sapiens.GRCh38.H1_hESC_3.Regulatory_Build.regulatory_activity.20221007.gff
Cleaning: /data/project/ubrite/drg-depot/Ensembl/rawdata/homo_sapiens/RegulatoryFeatureActivity/RegulatoryFeatureActivity/NK_PB/homo_sapiens.GRCh38.NK_PB.Regulatory_Build.regulatory_activity.20221007.gff
Cleaning: /data/project/ubrite/drg-depot/Ensembl/rawdata/homo_sapiens/RegulatoryFeatureActivity/RegulatoryFeatureActivity/naive_B_VB/homo_sapiens.GRCh38.naive_B_VB.Regulatory_Build.regulatory_activity.20221007.gff
Cleaning: /data/project/ubrite/drg-depot/Ensembl/rawdata/homo_sapiens/RegulatoryFeatureActivity/RegulatoryFeatureActivity/A549/homo_sapiens.GRCh38.A549.Regulatory_Bui